# Wildfire Data Pre-Processing

#### Data Sources:

(1)

Monitoring Trends in Burn Severity (MTBS)

Burned Areas Boundaries Dataset (1984-2022)

https://www.mtbs.gov/direct-download

(2)

Kaggle

1.88 Million US Wildfires

https://www.kaggle.com/datasets/rtatman/188-million-us-wildfires/data

In [1]:
import sqlite3
import pandas as pd
import numpy as np

## Load Data From Kaggle

In [2]:
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')

In [3]:
query = """
SELECT 
    FIRE_YEAR as ignition_year,
    STAT_CAUSE_DESCR as cause_description,
    LATITUDE as latitude,
    LONGITUDE as longitude,
    STATE as state,
    MTBS_ID as mtbs_id
FROM 
    'Fires'
WHERE
   MTBS_ID IS NOT NULL
"""

kaggle_df = pd.read_sql_query(query, cnx)

## Load Data From MTBS

In [4]:
import geopandas as gdp

In [5]:
mtbs_df_raw = gdp.read_file('data/mtbs/mtbs_perims_DD.shp')

In [6]:
mtbs_df = mtbs_df_raw.copy()

## Drop Unneeded Columns

In [7]:
mtbs_df = mtbs_df.drop(columns=[
    'Map_ID',
    'Map_Prog',
    'Asmnt_Type',
    'Pre_ID',
    'Post_ID',
    'Perim_ID',
    'dNBR_offst',
    'dNBR_stdDv',
    'NoData_T',
    'IncGreen_T',
    'Low_T',
    'Mod_T',
    'High_T',
    'Comment',
    'BurnBndLat',
    'BurnBndLon'
])

## Rename Columns

In [9]:
mtbs_df = mtbs_df.rename(columns={
            'Event_ID': 'mtbs_id',
            'irwinID': 'irwin_id',
            'Incid_Name': 'incident_name',
            'Incid_Type': 'incident_type',
            'BurnBndAc': 'burned_acreage',
            'Ig_Date': 'ignition_date'
})

## Remove Prescribed Burns

In [ ]:
df = df[df.incident_type != 'Prescribed Fire']

In [ ]:
df = df.drop(columns=['incident_type'])

## Merge Kaggle and MTBS Data

In [12]:
df = mtbs_df.merge(kaggle_df, on='mtbs_id', how='inner')

In [13]:
df.to_file('data/mtbs_kaggle_merged.geojson', driver='GeoJSON')  